In [9]:
from app.gen_planner.python.src import territory_splitter
import geopandas as gpd

zone = gpd.read_file('../scenarios_data/project Шлиссельбург.geojson')
zone_roads = gpd.read_file('../scenarios_data/roads_shlis.geojson')

# zone_roads.geometry = zone_roads.to_crs(32636).geometry.buffer(5)

zone_splitted = territory_splitter(zone, zone_roads)
zone_splitted.explore()

In [10]:
from app.gen_planner.python.src import GenPlanner, func_zones, terr_zones
from shapely import Point

zones_ratio_dict = {'right_lower': 0.225, 'right_upper': 0.225, 'left_upper': 0.225, 'left_lower': 0.225,
                    'middle_lower': 0.1, 'middle_upper': 0.1, 'free_zone1': 0.1, 'free_zone2': 0.1}
fixed_zones = gpd.GeoDataFrame()
middle_lower = Point(31.015, 59.911)
middle_upper = Point(31.022, 59.922)
right_upper = Point(31.046, 59.922)
right_lower = Point(31.043, 59.911)
left_upper = Point(30.999, 59.927)
left_lower = Point(30.993, 59.912)

fixed_2zones = gpd.GeoDataFrame(
    {'fixed_zone': [terr_zones.transport_terr, terr_zones.recreation_terr]},
    geometry=[left_lower, middle_upper],
    crs=4326)

fixed_4zones = gpd.GeoDataFrame(
    {'fixed_zone': [terr_zones.transport_terr, terr_zones.recreation_terr, terr_zones.special_terr, terr_zones.business_terr]},
    geometry=[right_upper, right_lower, left_upper, left_lower],
    crs=4326)

fixed_zones = fixed_4zones

# territory = zone.copy()
territory = zone_splitted.copy()

generated_zones, generated_roads = GenPlanner(territory, dev_mod=True).features2terr_zones(
    funczone=func_zones.residential_func_zone, fixed_terr_zones=fixed_zones)

2025-05-31 22:38:17.149 | INFO     | app.gen_planner.python.src.genplanner.genplanner:__init__:57 - Dev mod activated, no more ProcessPool


Статус: Optimal


In [11]:
import pandas as pd

m1 = pd.concat([generated_zones,
                fixed_zones.rename(columns={'fixed_zone': 'territory_zone'}).to_crs(generated_zones.crs)]).explore(
    tiles='Cartodb Positron', column='territory_zone', cmap='Dark2')
m1